In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [2]:
def split_dataset(x, y):
    y_5_or_6 = (y == 5) | (y == 6)
    y_a = y[~y_5_or_6]
    y_a[y_a > 6] -= 2
    y_b = (y[y_5_or_6] == 6).astype(np.float32)
    return ((x[~y_5_or_6], y_a), (x[y_5_or_6], y_b))

In [3]:
(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train_full = x_train_full / 255.0
x_test = x_test / 255.0
x_valid, x_train = x_train_full[:5000], x_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

(x_train_a, y_train_a), (x_train_b, y_train_b) = split_dataset(x_train, y_train)
(x_valid_a, y_valid_a), (x_valid_b, y_valid_b) = split_dataset(x_valid, y_valid)
(x_test_a, y_test_a), (x_test_b, y_test_b) = split_dataset(x_test, y_test)
x_train_b = x_train_b[:200]
y_train_b = y_train_b[:200]

In [4]:
x_train_a.shape

(43986, 28, 28)

In [5]:
x_train_b.shape

(200, 28, 28)

In [6]:
y_train_a[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [7]:
y_train_b[:30]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

In [8]:
tf.random.set_seed(42)
np.random.seed(42)

In [9]:
model_a = keras.models.Sequential()
model_a.add(keras.layers.Flatten(input_shape=[28, 28]))

for n_hidden in (300, 100, 50, 50, 50):
    model_a.add(keras.layers.Dense(n_hidden, activation='selu'))

model_a.add(keras.layers.Dense(8, activation='softmax'))

In [10]:
model_a.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 4

In [11]:
model_a.compile(loss='sparse_categorical_crossentropy', 
                optimizer=keras.optimizers.SGD(learning_rate=1e-3), 
                metrics=['acc'])

In [12]:
history = model_a.fit(x_train_a, y_train_a, epochs=20, 
                      validation_data=(x_valid_a, y_valid_a))

Epoch 1/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.5927 - acc: 0.8104 - val_loss: 0.3896 - val_acc: 0.8667
Epoch 2/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.3523 - acc: 0.8786 - val_loss: 0.3289 - val_acc: 0.8824
Epoch 3/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.3170 - acc: 0.8894 - val_loss: 0.3012 - val_acc: 0.8996
Epoch 4/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.2973 - acc: 0.8975 - val_loss: 0.2891 - val_acc: 0.9028
Epoch 5/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.2835 - acc: 0.9020 - val_loss: 0.2774 - val_acc: 0.9061
Epoch 6/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.2729 - acc: 0.9060 - val_loss: 0.2733 - val_acc: 0.9066
Epoch 7/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.2641 - acc: 0.9091 - val_loss: 0.2721 - val_acc: 0.9086
Epoch 8/20
1375/1375 [==============================] - 2s 2ms

In [13]:
model_a.save('model_a.h5')

In [14]:
model_b = keras.models.Sequential()
model_b.add(keras.layers.Flatten(input_shape=[28, 28]))

for n_hidden in (300, 100, 50, 50, 50):
    model_b.add(keras.layers.Dense(n_hidden, activation='selu'))

model_b.add(keras.layers.Dense(1, activation='sigmoid'))

In [15]:
model_b.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_7 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_8 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_9 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_10 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                

In [16]:
model_b.compile(loss='binary_crossentropy', 
                optimizer=keras.optimizers.SGD(learning_rate=1e-3), 
                metrics=['acc'])

In [17]:
history = model_b.fit(x_train_b, y_train_b, epochs=20, 
                      validation_data=(x_valid_b, y_valid_b))

Epoch 1/20
7/7 [==============================] - 0s 25ms/step - loss: 0.9573 - acc: 0.4650 - val_loss: 0.6314 - val_acc: 0.6004
Epoch 2/20
7/7 [==============================] - 0s 8ms/step - loss: 0.5692 - acc: 0.7450 - val_loss: 0.4784 - val_acc: 0.8529
Epoch 3/20
7/7 [==============================] - 0s 8ms/step - loss: 0.4503 - acc: 0.8650 - val_loss: 0.4102 - val_acc: 0.8945
Epoch 4/20
7/7 [==============================] - 0s 7ms/step - loss: 0.3879 - acc: 0.8950 - val_loss: 0.3647 - val_acc: 0.9178
Epoch 5/20
7/7 [==============================] - 0s 8ms/step - loss: 0.3435 - acc: 0.9250 - val_loss: 0.3300 - val_acc: 0.9320
Epoch 6/20
7/7 [==============================] - 0s 7ms/step - loss: 0.3081 - acc: 0.9300 - val_loss: 0.3019 - val_acc: 0.9402
Epoch 7/20
7/7 [==============================] - 0s 8ms/step - loss: 0.2800 - acc: 0.9350 - val_loss: 0.2804 - val_acc: 0.9422
Epoch 8/20
7/7 [==============================] - 0s 8ms/step - loss: 0.2564 - acc: 0.9450 - val_loss: 

In [18]:
model_a = keras.models.load_model('model_a.h5')
model_b_on_a = keras.models.Sequential(model_a.layers[:-1])
model_b_on_a.add(keras.layers.Dense(1, activation='sigmoid'))

In [19]:
model_a_clone = keras.models.clone_model(model_a)
model_a_clone.set_weights(model_a.get_weights())

In [20]:
for layer in model_b_on_a.layers[:-1]:
    layer.trainable = False

model_b_on_a.compile(loss='binary_crossentropy', 
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3), 
                     metrics=['acc'])

In [21]:
history = model_b_on_a.fit(x_train_b, y_train_b, epochs=4, 
                           validation_data=(x_valid_b, y_valid_b))

for layer in model_b_on_a.layers[:-1]:
    layer.trainable = True

model_b_on_a.compile(loss='binary_crossentropy', 
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3), 
                     metrics=['acc'])

history = model_b_on_a.fit(x_train_b, y_train_b, epochs=16, 
                           validation_data=(x_valid_b, y_valid_b))

Epoch 1/4
7/7 [==============================] - 0s 21ms/step - loss: 0.5731 - acc: 0.6550 - val_loss: 0.5783 - val_acc: 0.6359
Epoch 2/4
7/7 [==============================] - 0s 7ms/step - loss: 0.5371 - acc: 0.6950 - val_loss: 0.5413 - val_acc: 0.6846
Epoch 3/4
7/7 [==============================] - 0s 7ms/step - loss: 0.5008 - acc: 0.7300 - val_loss: 0.5096 - val_acc: 0.7140
Epoch 4/4
7/7 [==============================] - 0s 7ms/step - loss: 0.4697 - acc: 0.7550 - val_loss: 0.4813 - val_acc: 0.7363
Epoch 1/16
7/7 [==============================] - 0s 21ms/step - loss: 0.3924 - acc: 0.8150 - val_loss: 0.3438 - val_acc: 0.8661
Epoch 2/16
7/7 [==============================] - 0s 7ms/step - loss: 0.2778 - acc: 0.9350 - val_loss: 0.2590 - val_acc: 0.9290
Epoch 3/16
7/7 [==============================] - 0s 7ms/step - loss: 0.2071 - acc: 0.9650 - val_loss: 0.2103 - val_acc: 0.9554
Epoch 4/16
7/7 [==============================] - 0s 8ms/step - loss: 0.1663 - acc: 0.9800 - val_loss: 0.1

In [22]:
model_b.evaluate(x_test_b, y_test_b)

63/63 [==============================] - 0s 823us/step - loss: 0.1408 - acc: 0.9705


[0.14084076881408691, 0.9704999923706055]

In [23]:
model_b_on_a.evaluate(x_test_b, y_test_b)

63/63 [==============================] - 0s 802us/step - loss: 0.0681 - acc: 0.9930


[0.06810946017503738, 0.9929999709129333]

In [24]:
(100 - 97.05) / (100 - 99.3)

4.214285714285701